In [1]:
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
import os
import dotenv

In [2]:
dotenv.load_dotenv()

True

In [14]:
df_dwd = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',
                 sep=';',
                 parse_dates=['MESS_DATUM','MESS_DATUM_WOZ'],
                 date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H:%M"))
display(df.head, df.dtypes)

/var/folders/c5/491yhz3s1y93gvqdkz7t62p00000gn/T/ipykernel_37260/2147556409.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_dwd = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',


<bound method NDFrame.head of       mRID          Startzeit            Endzeit Resolution Position  Preis
0        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        1  22.40
1        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        2  -7.40
2        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        3 -10.16
3        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        4  -5.30
4        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        5   5.54
...    ...                ...                ...        ...      ...    ...
50039  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       90   9.61
50040  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       91   4.50
50041  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       92   0.10
50042  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       93   1.14
50043  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       94   0.00

[50044 rows x 6 columns]>

mRID           object
Startzeit      object
Endzeit        object
Resolution     object
Position       object
Preis         float64
dtype: object

In [307]:
# Abfrage der Daten auf entso-e
# Ausfühle Informationen unter https://documenter.getpostman.com/view/7009892/2s93JtP3F6#3b383df0-ada2-49fe-9a50-98b1bb201c6b

energy_prices_url = 'https://web-api.tp.entsoe.eu/api?'

response = requests.get(url=energy_prices_url,
                        params=
                        {'securityToken': os.getenv('TOKEN'),
                         'DocumentType': 'A44',
                         'ProcessType':'A01',
                         'periodStart': '202501010000',
                         'periodEnd': '202512310000',
                         'outBiddingZone_Domain': '10Y1001A1001A82H',
                         'out_Domain': '10Y1001A1001A82H',
                         'in_Domain': '10Y1001A1001A82H',
                         'contract_MarketAgreement.type': 'A01'
                        })

# Antwort verarbeiten
if response.status_code == 200:
    # Direkte Nutzung der XML-Daten (ohne Dekomprimierung)
    xml_data = response.content.decode('utf-8')  # Bytes zu String dekodieren

    # XML-Daten parsen
    root = ET.fromstring(xml_data)

    # Namespace für die XML-Daten (falls vorhanden)
    namespaces = {
        'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'
    }

    # Liste für die Daten
    data = []

    # Durch alle TimeSeries iterieren
    for timeseries in root.findall('.//ns:TimeSeries', namespaces):
        # Sequenz prüfen und auf 2 Filtern (finale Daten)
        sequence = timeseries.find('.//ns:classificationSequence_AttributeInstanceComponent.position', namespaces)
        if sequence is not None and sequence.text == '2':
            mrid = timeseries.find('ns:mRID', namespaces).text
            for period in timeseries.findall('.//ns:Period', namespaces):
                end = period.find('.//ns:timeInterval/ns:end', namespaces).text
                resolution = period.find('ns:resolution', namespaces).text
                for point in period.findall('.//ns:Point', namespaces):
                    position = point.find('ns:position', namespaces).text
                    price = point.find('ns:price.amount', namespaces).text

                    # Endzeit als Pandas-Datetime-Objekt speichern
                    end_datetime = pd.to_datetime(end).normalize()  # Uhrzeit auf 00:00:00 

                    data.append({
                        'mRID': mrid,
                        'Datum': end_datetime,  # Als Pandas-Datetime
                        'Resolution': resolution,
                        'Position': position,
                        'Preis': float(price)
                    })

    # DataFrame erstellen
    df = pd.DataFrame(data)

    # Ausgabe des DataFrames
    print(df)

    # Optional: DataFrame in CSV speichern
    #df.to_csv('day_ahead_preise_sequence2.csv', index=False)
else:
    print(f"Fehler bei der API-Anfrage: {response.status_code}")
    print(response.content)

      mRID                     Datum Resolution Position  Preis
0        1 2025-01-01 00:00:00+00:00      PT15M        1  22.40
1        1 2025-01-01 00:00:00+00:00      PT15M        2  -7.40
2        1 2025-01-01 00:00:00+00:00      PT15M        3 -10.16
3        1 2025-01-01 00:00:00+00:00      PT15M        4  -5.30
4        1 2025-01-01 00:00:00+00:00      PT15M        5   5.54
...    ...                       ...        ...      ...    ...
34605  729 2025-12-31 00:00:00+00:00      PT15M       92  68.40
34606  729 2025-12-31 00:00:00+00:00      PT15M       93  79.33
34607  729 2025-12-31 00:00:00+00:00      PT15M       94  73.85
34608  729 2025-12-31 00:00:00+00:00      PT15M       95  66.00
34609  729 2025-12-31 00:00:00+00:00      PT15M       96  57.85

[34610 rows x 5 columns]


In [308]:
print(df.loc[:,'Position'].unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100']


In [309]:
df[df['mRID']=='597']

,mRID,Datum,Resolution,Position,Preis
28352,597,2025-10-26 00:00:00+00:00,PT15M,1,22.21
28353,597,2025-10-26 00:00:00+00:00,PT15M,2,24.80
28354,597,2025-10-26 00:00:00+00:00,PT15M,3,13.20
28355,597,2025-10-26 00:00:00+00:00,PT15M,4,7.80
28356,597,2025-10-26 00:00:00+00:00,PT15M,5,15.40
...,...,...,...,...,...
28447,597,2025-10-26 00:00:00+00:00,PT15M,96,2.20
28448,597,2025-10-26 00:00:00+00:00,PT15M,97,19.30
28449,597,2025-10-26 00:00:00+00:00,PT15M,98,10.70
28450,597,2025-10-26 00:00:00+00:00,PT15M,99,7.43


In [310]:
# Anzeige der Datentypen
print(df.dtypes)

# Umwandeln von Position in Int Werte
df['Position'] = df['Position'].astype('Int64')

# Check Datentypen
print(df.dtypes)
print(f"Anzahl der Zeilen vor der Umwandlung: {len(df)}")

mRID                       object
Datum         datetime64[ns, UTC]
Resolution                 object
Position                   object
Preis                     float64
dtype: object
mRID                       object
Datum         datetime64[ns, UTC]
Resolution                 object
Position                    Int64
Preis                     float64
dtype: object
Anzahl der Zeilen vor der Umwandlung: 34610


In [311]:
df[df['mRID']=='597']

,mRID,Datum,Resolution,Position,Preis
28352,597,2025-10-26 00:00:00+00:00,PT15M,1,22.21
28353,597,2025-10-26 00:00:00+00:00,PT15M,2,24.80
28354,597,2025-10-26 00:00:00+00:00,PT15M,3,13.20
28355,597,2025-10-26 00:00:00+00:00,PT15M,4,7.80
28356,597,2025-10-26 00:00:00+00:00,PT15M,5,15.40
...,...,...,...,...,...
28447,597,2025-10-26 00:00:00+00:00,PT15M,96,2.20
28448,597,2025-10-26 00:00:00+00:00,PT15M,97,19.30
28449,597,2025-10-26 00:00:00+00:00,PT15M,98,10.70
28450,597,2025-10-26 00:00:00+00:00,PT15M,99,7.43


In [312]:
# Entfernen der doppelten Werte zur Zeitumstellungsstunde und Anpassen der Position Werte

# Zeilen prüfen
print(f"Anzahl der Zeilen vor der Umwandlung: {len(df)}")

# 1. Zeilen mit mRID == '597' und Position in [8, 9, 10, 11] entfernen
mask = (df['mRID'] == '597') & (df['Position'].isin([8, 9, 10, 11]))
df.drop(df[mask].index, axis=0, inplace=True)

# 2. Für alle Positionen >= 12 **nur bei mRID == '597'**: 4 abziehen
mask_597 = df['mRID'] == '597'
df.loc[mask_597, 'Position'] = df.loc[mask_597, 'Position'].apply(
    lambda x: x - 4 if x >= 12 else x
)
# Zeilen prüfen
print(f"Anzahl der Zeilen nach der Umwandlung: {len(df)}")

# Ergebnis anzeigen
print(df[df['mRID'] == '597'])

Anzahl der Zeilen vor der Umwandlung: 34610
Anzahl der Zeilen nach der Umwandlung: 34606
      mRID                     Datum Resolution  Position  Preis
28352  597 2025-10-26 00:00:00+00:00      PT15M         1  22.21
28353  597 2025-10-26 00:00:00+00:00      PT15M         2  24.80
28354  597 2025-10-26 00:00:00+00:00      PT15M         3  13.20
28355  597 2025-10-26 00:00:00+00:00      PT15M         4   7.80
28356  597 2025-10-26 00:00:00+00:00      PT15M         5  15.40
...    ...                       ...        ...       ...    ...
28447  597 2025-10-26 00:00:00+00:00      PT15M        92   2.20
28448  597 2025-10-26 00:00:00+00:00      PT15M        93  19.30
28449  597 2025-10-26 00:00:00+00:00      PT15M        94  10.70
28450  597 2025-10-26 00:00:00+00:00      PT15M        95   7.43
28451  597 2025-10-26 00:00:00+00:00      PT15M        96   0.00

[96 rows x 5 columns]


In [313]:
print(f"Anzahl der Zeilen vor der Umwandlung: {len(df)}")

def fuelle_fehlende_positionen(mrid_df):
    # Sortiere nach 'Position'
    mrid_df = mrid_df.sort_values(by='Position').reset_index(drop=True)

    # Liste aller erwarteten Positionen (1 bis 96)
    erwartete_positionen = list(range(1, 97))

    # Finde fehlende Positionen
    vorhandene_positionen = mrid_df['Position'].tolist()
    fehlende_positionen = [p for p in erwartete_positionen if p not in vorhandene_positionen]

    # Füge fehlende Positionen ein
    for fehlende_pos in fehlende_positionen:
        # Finde die vorherige Position
        vorherige_pos = max([p for p in vorhandene_positionen if p < fehlende_pos], default=None)

        if vorherige_pos is not None:
            # Hole die Zeile der vorherigen Position
            vorherige_zeile = mrid_df[mrid_df['Position'] == vorherige_pos].iloc[0]

            # Erstelle eine neue Zeile mit der fehlenden Position und allen Werten der vorherigen Zeile
            neue_zeile = vorherige_zeile.copy()
            neue_zeile['Position'] = fehlende_pos

            # Füge die neue Zeile in den DataFrame ein
            mrid_df = pd.concat([
                mrid_df.iloc[:mrid_df[mrid_df['Position'] == vorherige_pos].index[0] + 1],
                pd.DataFrame([neue_zeile]),
                mrid_df.iloc[mrid_df[mrid_df['Position'] == vorherige_pos].index[0] + 1:]
            ]).reset_index(drop=True)

    return mrid_df

# Wende die Funktion auf jede `mRID` separat an
result_df = pd.DataFrame()
for mrid, group in df.groupby('Datum'):
    gefuellt_df = fuelle_fehlende_positionen(group)
    result_df = pd.concat([result_df, gefuellt_df])

print(f"Anzahl der Zeilen nach der Umwandlung: {len(df)}")
print(result_df)

Anzahl der Zeilen vor der Umwandlung: 34606
Anzahl der Zeilen nach der Umwandlung: 34606
   mRID                     Datum Resolution  Position  Preis
0     1 2025-01-01 00:00:00+00:00      PT15M         1  22.40
1     1 2025-01-01 00:00:00+00:00      PT15M         2  -7.40
2     1 2025-01-01 00:00:00+00:00      PT15M         3 -10.16
3     1 2025-01-01 00:00:00+00:00      PT15M         4  -5.30
4     1 2025-01-01 00:00:00+00:00      PT15M         5   5.54
..  ...                       ...        ...       ...    ...
91  729 2025-12-31 00:00:00+00:00      PT15M        92  68.40
92  729 2025-12-31 00:00:00+00:00      PT15M        93  79.33
93  729 2025-12-31 00:00:00+00:00      PT15M        94  73.85
94  729 2025-12-31 00:00:00+00:00      PT15M        95  66.00
95  729 2025-12-31 00:00:00+00:00      PT15M        96  57.85

[35040 rows x 5 columns]


In [314]:
print(f"Fehlende Werte in 'Datum': {result_df['Datum'].isna().sum()}")
print(f"Fehlende Werte in 'Position': {result_df['Position'].isna().sum()}")
print(f"Anzahl der Zeilen vor der Umwandlung: {len(result_df)}")

Fehlende Werte in 'Datum': 0
Fehlende Werte in 'Position': 0
Anzahl der Zeilen vor der Umwandlung: 35040


In [315]:
# Umwandeln der Daten in 15 minütige Datumsangaben mit den Preisen

# 1. 'Position' in Minuten umwandeln (Position 1 → 0 Minuten, Position 2 → 15 Minuten, usw.)
result_df['Position_Minuten'] = (result_df['Position'] - 1) * 15

# 2. Minuten als Zeitdifferenz zum 'Datum' hinzufügen
result_df['Datum_Uhrzeit'] = result_df['Datum'] + pd.to_timedelta(result_df['Position_Minuten'], unit='min')

# Position_Minuten löschen, falls nicht mehr benötigt
#df = df.drop(columns=['Position_Minuten'])

# Ergebnis anzeigen
print(result_df)

   mRID                     Datum Resolution  Position  Preis  \
0     1 2025-01-01 00:00:00+00:00      PT15M         1  22.40   
1     1 2025-01-01 00:00:00+00:00      PT15M         2  -7.40   
2     1 2025-01-01 00:00:00+00:00      PT15M         3 -10.16   
3     1 2025-01-01 00:00:00+00:00      PT15M         4  -5.30   
4     1 2025-01-01 00:00:00+00:00      PT15M         5   5.54   
..  ...                       ...        ...       ...    ...   
91  729 2025-12-31 00:00:00+00:00      PT15M        92  68.40   
92  729 2025-12-31 00:00:00+00:00      PT15M        93  79.33   
93  729 2025-12-31 00:00:00+00:00      PT15M        94  73.85   
94  729 2025-12-31 00:00:00+00:00      PT15M        95  66.00   
95  729 2025-12-31 00:00:00+00:00      PT15M        96  57.85   

    Position_Minuten             Datum_Uhrzeit  
0                  0 2025-01-01 00:00:00+00:00  
1                 15 2025-01-01 00:15:00+00:00  
2                 30 2025-01-01 00:30:00+00:00  
3                 45 20

In [316]:
from entsoe import EntsoePandasClient

client = EntsoePandasClient(api_key=os.getenv('TOKEN'))

start = pd.Timestamp('20250101', tz='Europe/Brussels')
end = pd.Timestamp('20260101', tz='Europe/Brussels')
country_code = 'DE_LU'  # Belgium
country_code_from = 'DE_LU'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A44'
resolution = '15min'
sequence = 2

# methods that return Pandas Series
price_series = client.query_day_ahead_prices_local(country_code=country_code,
                                    sequence=2,
                                    start=start,
                                    end=end,
                                    resolution='15min')


In [317]:
print(price_series.index)

DatetimeIndex(['2025-01-01 00:00:00+01:00', '2025-01-01 00:15:00+01:00',
               '2025-01-01 00:30:00+01:00', '2025-01-01 00:45:00+01:00',
               '2025-01-01 01:00:00+01:00', '2025-01-01 01:15:00+01:00',
               '2025-01-01 01:30:00+01:00', '2025-01-01 01:45:00+01:00',
               '2025-01-01 02:00:00+01:00', '2025-01-01 02:15:00+01:00',
               ...
               '2025-12-31 21:45:00+01:00', '2025-12-31 22:00:00+01:00',
               '2025-12-31 22:15:00+01:00', '2025-12-31 22:30:00+01:00',
               '2025-12-31 22:45:00+01:00', '2025-12-31 23:00:00+01:00',
               '2025-12-31 23:15:00+01:00', '2025-12-31 23:30:00+01:00',
               '2025-12-31 23:45:00+01:00', '2026-01-01 00:00:00+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', length=35040, freq=None)


In [322]:
df_new = price_series.reset_index()
df_new = df_new.rename(columns={'index':'Datum', 0:'Preis'})
df_new['Datum'] = df_new['Datum'].dt.tz_localize(None).dt.tz_localize('UTC')
print(df_new.dtypes)
print(df_new)

Datum    datetime64[ns, UTC]
Preis                float64
dtype: object
                          Datum  Preis
0     2025-01-01 00:00:00+00:00  22.40
1     2025-01-01 00:15:00+00:00  -7.40
2     2025-01-01 00:30:00+00:00 -10.16
3     2025-01-01 00:45:00+00:00  -5.30
4     2025-01-01 01:00:00+00:00   5.54
...                         ...    ...
35035 2025-12-31 23:00:00+00:00  79.33
35036 2025-12-31 23:15:00+00:00  73.85
35037 2025-12-31 23:30:00+00:00  66.00
35038 2025-12-31 23:45:00+00:00  57.85
35039 2026-01-01 00:00:00+00:00  51.28

[35040 rows x 2 columns]


In [323]:
result_df.dtypes

mRID                             object
Datum               datetime64[ns, UTC]
Resolution                       object
Position                          Int64
Preis                           float64
Position_Minuten                  Int64
Datum_Uhrzeit       datetime64[ns, UTC]
dtype: object

In [326]:
# DataFrames nach den Datumsangaben zusammenführen
df_vergleich = pd.merge(
    result_df[['Datum_Uhrzeit', 'Preis']],
    df_new[['Datum', 'Preis']],
    left_on='Datum_Uhrzeit',
    right_on='Datum',
    suffixes=('_result', '_new')
)

# Vergleich der Preise
df_vergleich['Preis_Vergleich'] = df_vergleich['Preis_result'] == df_vergleich['Preis_new']

# Nicht übereinstimmende Zeilen anzeigen
nicht_uebereinstimmend = df_vergleich[~df_vergleich['Preis_Vergleich']]

# Ausgabe der nicht übereinstimmenden Zeilen
print(nicht_uebereinstimmend)
nicht_uebereinstimmend.to_csv('Vergleich der Preise der Abrufe.csv')

                  Datum_Uhrzeit  Preis_result                     Datum  \
8456  2025-03-30 03:00:00+00:00         12.60 2025-03-30 03:00:00+00:00   
8457  2025-03-30 03:15:00+00:00         52.40 2025-03-30 03:15:00+00:00   
8458  2025-03-30 03:30:00+00:00         49.44 2025-03-30 03:30:00+00:00   
8459  2025-03-30 03:45:00+00:00         12.59 2025-03-30 03:45:00+00:00   
8460  2025-03-30 04:00:00+00:00         44.91 2025-03-30 04:00:00+00:00   
...                         ...           ...                       ...   
28611 2025-10-26 01:45:00+00:00         -7.90 2025-10-26 01:45:00+00:00   
28612 2025-10-26 02:00:00+00:00          4.90 2025-10-26 02:00:00+00:00   
28614 2025-10-26 02:15:00+00:00          1.90 2025-10-26 02:15:00+00:00   
28616 2025-10-26 02:30:00+00:00          1.80 2025-10-26 02:30:00+00:00   
28618 2025-10-26 02:45:00+00:00          1.90 2025-10-26 02:45:00+00:00   

       Preis_new  Preis_Vergleich  
8456       22.59            False  
8457       59.90           